<a href="https://colab.research.google.com/github/Roterdamjr/ControleFinanceiro/blob/main/Controle_Financeiro_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Biblioteca

> Adicionar aspas



In [57]:
!pip install xlsxwriter

In [58]:
import pandas as pd
import numpy as np
from io import BytesIO
import yfinance as yf

def load_data(excel_filepath):

    ########## historico ###############
    df_historico = pd.read_excel(
        excel_filepath,
        sheet_name='Historico',
        engine='openpyxl'
    )

    df_historico = df_historico.dropna(axis=1,how='all')
    df_historico.set_index('Ativo', inplace=True)

    ########## cadastro acoes ###############
    df_cadastro_acoes = pd.read_excel(
        excel_filepath,
        sheet_name='Cadastro Acoes',
        engine='openpyxl'
    )

    df_cadastro_acoes = df_cadastro_acoes.dropna(axis=1,how='all')
    df_cadastro_acoes.set_index('Ativo', inplace=True)

    ########## aportes ###############
    df_aportes = pd.read_excel(
        excel_filepath,
        sheet_name='Aportes',
        engine='openpyxl'
    )

    aporte_dh = df_aportes['DH'].iloc[0] if not df_aportes['DH'].empty else 0.0
    aporte_fii = df_aportes['FII'].iloc[0] if not df_aportes['FII'].empty else 0.0

    return df_historico, df_cadastro_acoes, aporte_dh, aporte_fii

def fn_busca_pm_quantidade(df):

  #recebe dados de uma ativo e deveolve preco médio, quantidade

  preco_medio, qtde_total = 0.0, 0

  for indice, linha in df.iterrows():
      valor = linha['Valor']
      qtde = linha['Quant']
      operacao = linha['Operação']

      if operacao == 'C':
          preco_medio = (preco_medio * qtde_total + valor * qtde) / (qtde_total + qtde)
          qtde_total += qtde
      elif operacao == 'V':
          qtde_total -= qtde
      elif operacao == 'A':
          preco_medio = (preco_medio * qtde_total) / (qtde_total - qtde)
          qtde_total -= qtde
      elif operacao == 'D':
          preco_medio = (preco_medio * qtde_total) / (qtde_total + qtde)
          qtde_total += qtde

  return qtde_total,preco_medio

def fn_calc_metricas(nome_ativo,
                     df_historico,
                     df_alocacao,
                     df_cotacoes ):

  ##############################
  # calcula quantidade, preco_medio, cotacao ,
  # patrimonio, resultado, rentabilidade, alocacao
  ##############################

  df_ativo = df_historico.loc[nome_ativo]
  quantidade, preco_medio = fn_busca_pm_quantidade(df_ativo)
  cotacao = df_cotacoes[nome_ativo]

  alocacao = df_alocacao.loc[nome_ativo]['Alocacao']
  patrimonio = quantidade * cotacao
  resultado = patrimonio - quantidade * preco_medio
  rentabilidade = resultado / (quantidade * preco_medio)

  return [int(quantidade),
          float(round(preco_medio, 2)),
          float(round(cotacao, 2)) ,
          float(round(patrimonio, 2)),
          float(round(resultado, 2)),
          float(round(rentabilidade, 2)),
          float(round(alocacao, 3))]

def fn_busca_cotacoes(p_ativos):
  df_cotacoes = pd.DataFrame()
  df_cotacoes = yf.download([at +".SA" for at in p_ativos])['Close'].iloc[-1]
  df_cotacoes.index = df_cotacoes.index.str.replace('.SA', '', regex=False)
  return df_cotacoes

def fn_object_para_numerico(df,nome_coluna):

#    coluna_limpa = df[nome_coluna].astype(str).str.replace('.', '', regex=False)
#    coluna_limpa = coluna_limpa.str.replace(',', '.', regex=False)
 #   df[nome_coluna] = pd.to_numeric(coluna_limpa, errors='coerce')

    df[nome_coluna] = df[nome_coluna].str.replace('.', '', regex=False)
    df[nome_coluna] = df[nome_coluna].str.replace(',', '.', regex=False).astype(float)


def fn_percet_para_numerico(df, nome_coluna):

  coluna_limpa = (
          df[nome_coluna]
          .astype(str)
          .str.replace('%', '', regex=False)
          .str.replace(',', '.', regex=False)
      )

  df['Alocacao'] = pd.to_numeric(coluna_limpa, errors='coerce') / 100


# Teste

In [ ]:
# Agrupamento 5x1

df_hist = pd.read_csv('/content/Teste- Historico.csv',encoding='latin-1', sep=';')
df_hist = df_hist.dropna(axis=1,how='all')
df_hist = df_hist.dropna(axis=0,how='all')
fn_object_para_numerico(df_hist,'Valor')
fn_object_para_numerico(df_hist,'Custos')

nome_ativo = 'ALOS3'
df_ativ = df_hist.loc[df_hist['Ativo'] == nome_ativo]
qtde_total,preco_medio = fn_metricas(df_ativ)
qtde_total, preco_medio

(120, 30.208994708994705)

In [15]:
import pandas as pd

# DataFrame de Exemplo
data = {'Produto': ['A', 'B', 'C'],
        'Valor': ['1.223,50', '20,90', '5,00']}
df = pd.DataFrame(data)

# 1. Substitui a vírgula pelo ponto e 2. Converte para float
df['Valor'] = df['Valor'].str.replace('.', '', regex=False)
df['Valor'] = df['Valor'].str.replace(',', '.', regex=False).astype(float)
df

,Produto,Valor
0,A,1223.5
1,B,20.9
2,C,5.0


# Execução

In [59]:
df_historico, df_cadastro_acoes, aporte_dh, aporte_fii = load_data('/content/Controle Financeiro.xlsx' )


df_cotacoes = fn_busca_cotacoes(df_cadastro_acoes.index.tolist())

/tmp/ipython-input-779408974.py:94: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_cotacoes = yf.download([at +".SA" for at in p_ativos])['Close'].iloc[-1]
[*********************100%***********************]  17 of 17 completed


In [60]:
## popula carteira com todas as métricas

df_carteira = pd.DataFrame([fn_calc_metricas(ativo, df_historico, df_cadastro_acoes, df_cotacoes)
                              for ativo in df_cadastro_acoes.index],
                          columns=['Quantidade', 'Preço Médio', 'Cotação', 'Patrimônio', 'Resultado', 'Rentabilidade', 'Alocacao'],
                          index=df_cadastro_acoes.index)

df_carteira['Preco Justo'] = df_cadastro_acoes['Preco Justo']

# Totais
patrimonio_total, resultado_total = df_carteira['Patrimônio'].sum(), df_carteira['Resultado'].sum()
df_carteira['A Aportar'] = ( df_carteira['Alocacao'] * (patrimonio_total + aporte_dh) - df_carteira['Patrimônio'] )
df_carteira

,Quantidade,Preço Médio,Cotação,Patrimônio,Resultado,Rentabilidade,Alocacao,Preco Justo,A Aportar
Ativo,,,,,,,,,
ABCB4,670,19.94,24.45,16381.50,3018.67,0.23,0.081,25.00,3274.54563
ALOS3,430,22.07,28.35,12190.50,2701.53,0.28,0.023,27.00,-6609.15371
BBAS3,480,26.26,22.47,10785.60,-1818.59,-0.14,0.000,20.00,-10785.60000
BBDC4,900,13.44,19.65,17685.00,5593.00,0.46,0.068,20.00,-1183.62836
CSMG3,300,22.01,41.63,12489.00,5887.29,0.89,0.081,25.00,7167.04563
CXSE3,1300,14.10,16.19,21047.00,2722.46,0.15,0.108,17.50,5161.06084
EGIE3,90,41.16,30.64,2757.60,-946.68,-0.26,0.029,42.00,4279.74967
FLRY3,300,12.95,16.05,4815.00,931.00,0.24,0.029,14.50,2222.34967
ITSA4,1636,9.95,12.33,20171.88,3900.73,0.24,0.066,11.00,-4155.84282


In [53]:
patrimonio_total,aporte_dh

(np.float64(222667.22999999998), np.int64(20000))

# Exportação

In [ ]:

df = df_carteira
# -----------------------------------------------------

# 2. Defina o nome do arquivo e crie o ExcelWriter
nome_arquivo = 'relatorio_formatado.xlsx'
writer = pd.ExcelWriter(nome_arquivo, engine='xlsxwriter')

sheet_name = 'Relatório de Investimentos'

# Exporte o DataFrame para a aba, incluindo o índice (Ativo)
# startrow=1 (começa na linha 2), header=False, index=True (inclui a coluna de índice)
df.to_excel(writer, sheet_name=sheet_name, startrow=1, header=False, index=True)

# 3. Acesse o workbook e a worksheet
workbook = writer.book
worksheet = writer.sheets[sheet_name]

# 4. Defina os formatos de cores e números
# Cores RGB em hexadecimal
COR_CINZA = '#A9A9A9'
COR_BEGE_CLARO = '#F5F5DC'
COR_VERDE = '#90EE90'
COR_BEGE_PADRAO = '#F0EAD6'

# Formatos de números (Criação antecipada)
formato_moeda = workbook.add_format({'num_format': 'R$ #,##0.00'})
formato_porcentagem = workbook.add_format({'num_format': '0.00%'})

# Formato do Cabeçalho (Cinza)
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color': COR_CINZA,
    'border': 1
})

# Formato da coluna 'Ativo' (Coluna do Índice)
ativo_format = workbook.add_format({'fg_color': COR_BEGE_CLARO, 'border': 1})

# Formato da coluna 'A Aportar' (Verde com formato Moeda)
aportar_format = workbook.add_format({'fg_color': COR_VERDE, 'border': 1, 'num_format': 'R$ #,##0.00'})

# Formato das Colunas Restantes (Bege Padrão)
resto_format = workbook.add_format({'fg_color': COR_BEGE_PADRAO, 'border': 1})
resto_moeda_format = workbook.add_format({'fg_color': COR_BEGE_PADRAO, 'border': 1, 'num_format': 'R$ #,##0.00'})
resto_percent_format = workbook.add_format({'fg_color': COR_BEGE_PADRAO, 'border': 1, 'num_format': '0.00%'})


# 5. Aplique os Formatos
# O cabeçalho inclui o nome do índice (Ativo) na primeira coluna (Coluna 0).
colunas_excel = ['Ativo'] + list(df.columns.values)

# Escreva o Cabeçalho (na linha 1, pois o DataFrame começa na linha 2 com startrow=1)
for col_num, value in enumerate(colunas_excel):
    worksheet.write(0, col_num, value, header_format)
    # Adiciona a formatação de tamanho nas colunas
    if value in ['Ativo', 'Tipo']:
        worksheet.set_column(col_num, col_num, 10)
    elif value in ['Quantidade', 'Preço Médio', 'Cotação']:
        worksheet.set_column(col_num, col_num, 15)
    else:
        worksheet.set_column(col_num, col_num, 12)

# Mapeamento de colunas para o formato específico
formatos_especificos = {
    'Ativo': ativo_format,
    'Preço Médio': resto_moeda_format,
    'Cotação': resto_moeda_format,
    'Patrimônio': resto_moeda_format,
    'Resultado': resto_moeda_format,
    'Rentabilidade': resto_percent_format,
    'Alocacao': resto_percent_format,
    'A Aportar': aportar_format,
}


num_linhas = len(df)
# Itera sobre todas as linhas do DataFrame (começando da linha 1 do Excel)
# row = Linha do Excel (0 é o cabeçalho)
for row in range(1, num_linhas + 1):

    # Itera sobre todas as COLUNAS do Excel (incluindo o índice)
    for col_num, col_name in enumerate(colunas_excel):

        # 1. Obter o valor
        if col_name == 'Ativo':
            # Valor do índice (Ativo)
            valor = df.index[row - 1]
        else:
            # Valor da coluna de dados
            valor = df.iloc[row - 1][col_name]

        # 2. Obter o formato (Se não tiver formato específico, usa o resto_format)
        formato = formatos_especificos.get(col_name, resto_format)

        # 3. Escrever na célula
        worksheet.write(row, col_num, valor, formato)

# 6. Salve o arquivo
writer.close()

print(f"Arquivo '{nome_arquivo}' exportado com sucesso com a formatação de cores e números!")

Arquivo 'relatorio_formatado.xlsx' exportado com sucesso com a formatação de cores e números!
